In [10]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from prophet import Prophet
from tqdm import tqdm
import pickle
import os
from datetime import datetime
import json

# Define consistent colors for plots
TRAIN_COLOR = 'steelblue'
TRAIN_FILL_COLOR = 'steelblue'
TRAIN_FILL_ALPHA = 0.3
VAL_COLOR = 'coral'
VAL_FILL_COLOR = 'coral'
VAL_FILL_ALPHA = 0.3
TEST_COLOR = 'forestgreen'
TEST_FILL_COLOR = 'forestgreen'
TEST_FILL_ALPHA = 0.3

# Set random seeds for reproducibility
def set_seeds(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    
set_seeds()

## Data Loading and Preprocessing

In [11]:
def load_data(file_path):
    data = pd.read_parquet(file_path)
    return data

def check_for_missing_values(data):
    missing_values = data.isnull().sum()
    if missing_values.any():
        print("Missing values found in the dataset:")
        print(missing_values[missing_values > 0])
    else:
        print("No missing values found in the dataset.")
    return missing_values

def split_data(data, train_years, val_year, test_year):
    data['time_bucket'] = pd.to_datetime(data['time_bucket'])
    
    train_data = data[data['time_bucket'].dt.year.isin(train_years)]
    val_data = data[data['time_bucket'].dt.year == val_year]
    test_data = data[data['time_bucket'].dt.year == test_year]
    
    print(f"Train data size: {len(train_data)}")
    print(f"Validation data size: {len(val_data)}")
    print(f"Test data size: {len(test_data)}")
    
    return train_data, val_data, test_data

def filter_ride_data(data, ride_name):
    return data[data[f'ride_name_{ride_name}'] == True].copy()

def get_all_rides(data):
    ride_columns = [col for col in data.columns if col.startswith('ride_name_')]
    return [col.replace('ride_name_', '') for col in ride_columns]

def filter_to_operating_hours(ride_data):
    # Determine operating hours from data where wait times > 0
    operating_hours = ride_data[ride_data["wait_time"] > 0].groupby(
        ride_data["timestamp"].dt.date
    )["timestamp"].agg(['min', 'max']).reset_index()
    
    # Extract opening and closing hours
    operating_hours['opening_hour'] = pd.to_datetime(operating_hours['min']).dt.hour
    operating_hours['closing_hour'] = pd.to_datetime(operating_hours['max']).dt.hour
    
    # Set reasonable boundaries for operating hours
    operating_hours['opening_hour'] = operating_hours['opening_hour'].clip(lower=9, upper=11)
    operating_hours['closing_hour'] = operating_hours['closing_hour'].clip(lower=17, upper=21)
    
    # Create date-to-hours mapping
    date_to_hours = {}
    for _, row in operating_hours.iterrows():
        date_to_hours[row['timestamp']] = (row['opening_hour'], row['closing_hour'])
    
    # Filter data to operating hours only
    def is_operating_hour(timestamp):
        date = timestamp.date()
        if date not in date_to_hours:
            # Return 0 for dates not in mapping (likely closed days)
            return 0
        
        open_hour, close_hour = date_to_hours[date]
        hour = timestamp.hour
        return 1 if open_hour <= hour < close_hour else 0
    
    ride_data['operating_hour'] = ride_data['timestamp'].apply(is_operating_hour)
    ride_data = ride_data[ride_data['operating_hour'] == 1]
    ride_data = ride_data.drop(columns=["operating_hour"])
    
    return ride_data

## Model Loading and Prediction Classes

In [12]:
class BaseTimeSeriesModel:
    def __init__(self):
        self.model = None
        self.forecast = None
        self.holidays = None
        
    def prepare_prophet_dataframe(self, data, include_y=True):
        prophet_df = data[['timestamp', 'wait_time', 'temperature_unscaled', 'rain_unscaled', 'is_weekend']].copy()
        prophet_df = prophet_df.rename(columns={'timestamp': 'ds', 'wait_time': 'y', 'temperature_unscaled': 'temperature', 'rain_unscaled': 'rain'})

        if not include_y:
            prophet_df = prophet_df.drop(["y"], axis=1) 
        
        # Add additional features
        prophet_df['temp_squared'] = prophet_df['temperature'] ** 2
        prophet_df['high_temp'] = (prophet_df['temperature'] > 25).astype(int)
        prophet_df['any_rain'] = (prophet_df['rain'] > 0).astype(int)
        prophet_df['temp_weekend'] = prophet_df['temperature'] * prophet_df['is_weekend']
        prophet_df['rain_weekend'] = prophet_df['rain'] * prophet_df['is_weekend']
        
        return prophet_df
    
    def predict(self, future_df):
        # Add required columns for prediction if they're not already present
        future_df = future_df.copy()
        
        # Add COVID regressors if they're not already present
        if 'between_covid_lockdowns' not in future_df.columns:
            future_df['between_covid_lockdowns'] = 0
            covid_period = (future_df['ds'] >= '2020-05-20') & (future_df['ds'] <= '2020-11-01')
            future_df.loc[covid_period, 'between_covid_lockdowns'] = 1
            
        if 'covid_recovery' not in future_df.columns:
            future_df['covid_recovery'] = 0
            recovery_period = (future_df['ds'] >= '2021-05-21') & (future_df['ds'] <= '2021-08-31')
            future_df.loc[recovery_period, 'covid_recovery'] = 1
        
        # Make predictions
        self.forecast = self.model.predict(future_df)
        
        # Apply post-processing
        self.forecast = self.post_process_forecast(self.forecast)
        
        return self.forecast
    
    def post_process_forecast(self, forecast):
        forecast = forecast.copy()
        
        # Correct negative predictions
        negative_mask = forecast['yhat'] < 0
        forecast.loc[negative_mask, 'yhat'] = 0
        forecast.loc[negative_mask, 'yhat_lower'] = 0
        forecast.loc[negative_mask, 'yhat_upper'] = 0
        
        return forecast

## Model Evaluation Functions

In [13]:
def evaluate_model(ride_df, actual_values, predictions, title=""):
    # Calculate metrics
    mae = np.mean(np.abs(predictions - actual_values))
    rmse = np.sqrt(np.mean(np.square(predictions - actual_values)))
    
    # For sMAPE, avoid division by zero
    epsilon = 1e-8
    abs_pct_errors = np.abs(predictions - actual_values) / (np.abs(predictions) + np.abs(actual_values) + epsilon)
    # Only include points where actual values are non-zero
    non_zero_mask = (actual_values > 0) & (predictions > 0)
    smape = np.mean(abs_pct_errors[non_zero_mask]) * 100

    # Calculate additional metrics for test evaluation
    mse = np.mean(np.square(predictions - actual_values))
    
    # R-squared
    ss_res = np.sum((actual_values - predictions) ** 2)
    ss_tot = np.sum((actual_values - np.mean(actual_values)) ** 2)
    r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0

    # Print metrics
    print(f"\n{title} MAE: {mae:.2f} minutes")
    print(f"{title} RMSE: {rmse:.2f} minutes")
    print(f"{title} MSE: {mse:.2f}")
    print(f"{title} sMAPE: {smape:.2f}%")
    
    # Create a DataFrame with results for time-based analysis
    results_df = pd.DataFrame({
        'time_bucket': ride_df['time_bucket'].values,
        'actual': actual_values,
        'predicted': predictions,
    })
    
    # Add time components
    results_df['hour'] = results_df['time_bucket'].dt.hour
    results_df['day_of_week'] = results_df['time_bucket'].dt.dayofweek
    results_df['month'] = results_df['time_bucket'].dt.month
    results_df['date'] = results_df['time_bucket'].dt.date
    
    # Calculate errors
    results_df['error'] = results_df['predicted'] - results_df['actual']
    results_df['abs_error'] = np.abs(results_df['error'])
    results_df['pct_error'] = abs_pct_errors * 100
    
    # Visualize results
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Actual vs Predicted scatter plot
    axes[0,0].scatter(actual_values, predictions, alpha=0.5, color=TEST_COLOR)
    max_val = max(np.max(actual_values), np.max(predictions))
    axes[0,0].plot([0, max_val], [0, max_val], 'k--')
    axes[0,0].set_xlabel('Actual Wait Time (minutes)')
    axes[0,0].set_ylabel('Predicted Wait Time (minutes)')
    axes[0,0].set_title(f'{title} - Actual vs Predicted')
    axes[0,0].grid(True, alpha=0.3)
    
    # Error distribution
    axes[0,1].hist(results_df['error'], bins=30, alpha=0.7, color=TEST_COLOR, edgecolor='black')
    axes[0,1].axvline(x=0, color='red', linestyle='--', alpha=0.7)
    axes[0,1].set_xlabel('Prediction Error (Predicted - Actual)')
    axes[0,1].set_ylabel('Frequency')
    axes[0,1].set_title(f'{title} - Error Distribution')
    axes[0,1].grid(True, alpha=0.3)
    
    # Hourly analysis
    hourly_errors = results_df.groupby('hour')['abs_error'].mean()
    hourly_errors.plot(kind='bar', ax=axes[1,0], color=TEST_COLOR, alpha=0.7)
    axes[1,0].set_xlabel('Hour of Day')
    axes[1,0].set_ylabel('Mean Absolute Error (minutes)')
    axes[1,0].set_title(f'{title} - Error Analysis by Hour of Day')
    axes[1,0].grid(True, alpha=0.3)
    
    # Time series plot (sample of data)
    if len(results_df) > 1000:
        sample_df = results_df.sample(n=1000).sort_values('time_bucket')
    else:
        sample_df = results_df.sort_values('time_bucket')
    
    axes[1,1].plot(sample_df['time_bucket'], sample_df['actual'], 
                   label='Actual', alpha=0.7, color='blue')
    axes[1,1].plot(sample_df['time_bucket'], sample_df['predicted'], 
                   label='Predicted', alpha=0.7, color=TEST_COLOR)
    axes[1,1].set_xlabel('Date')
    axes[1,1].set_ylabel('Wait Time (minutes)')
    axes[1,1].set_title(f'{title} - Time Series (Sample)')
    axes[1,1].legend()
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Create metrics dictionary
    metrics = {
        "mae": mae,
        "rmse": rmse,
        "mse": mse,
        "smape": smape,
        "r2": r2
    }
    
    return metrics, results_df, fig

def evaluate_confidence_intervals(actual_values, lower_bounds, upper_bounds, predictions):
    # Coverage rate (percentage of actual values within confidence intervals)
    within_ci = (actual_values >= lower_bounds) & (actual_values <= upper_bounds)
    coverage_rate = np.mean(within_ci) * 100
    
    # Average interval width
    avg_interval_width = np.mean(upper_bounds - lower_bounds)
    
    # Interval score (lower is better)
    alpha = 0.05  # For 95% confidence intervals
    interval_score = np.mean(
        (upper_bounds - lower_bounds) + 
        (2/alpha) * np.maximum(0, lower_bounds - actual_values) +
        (2/alpha) * np.maximum(0, actual_values - upper_bounds)
    )
    
    print(f"Confidence Interval Evaluation:")
    print(f"Coverage Rate: {coverage_rate:.1f}% (target: 95%)")
    print(f"Average Interval Width: {avg_interval_width:.2f} minutes")
    print(f"Interval Score: {interval_score:.2f}")
    
    return {
        "coverage_rate": coverage_rate,
        "avg_interval_width": avg_interval_width,
        "interval_score": interval_score
    }

## Model Storage Functions

In [14]:
def load_model(ride_name, output_dir="models"):
    # Create ride-specific directory path
    ride_dir = os.path.join(output_dir, ride_name.replace(" ", "_"))
    
    # Check if models exist
    if not os.path.exists(ride_dir):
        return None, None
    
    # Load Prophet model
    try:
        with open(os.path.join(ride_dir, "prophet_model.pkl"), "rb") as f:
            prophet_model_obj = pickle.load(f)
    except:
        print(f"Could not load model for {ride_name}")
        return None, None
    
    # Initialize BaseTimeSeriesModel and set the loaded model
    prophet_ts = BaseTimeSeriesModel()
    prophet_ts.model = prophet_model_obj
    
    # Load holidays if they exist
    holidays_path = os.path.join(ride_dir, "holidays.csv")
    if os.path.exists(holidays_path):
        prophet_ts.holidays = pd.read_csv(holidays_path)
        prophet_ts.holidays['ds'] = pd.to_datetime(prophet_ts.holidays['ds'])
    
    # Load metrics
    with open(os.path.join(ride_dir, "metrics.json"), "r") as f:
        metrics = json.load(f)
    
    return prophet_ts, metrics

def get_processed_rides(output_dir="models"):
    if not os.path.exists(output_dir):
        return []
    
    # Get all subdirectories in the output directory that contain a model file
    processed_rides = []
    for d in os.listdir(output_dir):
        dir_path = os.path.join(output_dir, d)
        if os.path.isdir(dir_path):
            model_path = os.path.join(dir_path, "prophet_model.pkl")
            if os.path.exists(model_path):
                processed_rides.append(d)
    
    # Convert directory names back to ride names
    processed_rides = [ride.replace("_", " ") for ride in processed_rides]
    
    return processed_rides

def save_test_results(ride_name, test_metrics, ci_metrics, results_df, fig, output_dir="models"):
    # Create ride-specific directory
    ride_dir = os.path.join(output_dir, ride_name.replace(" ", "_"))
    
    # Load existing metrics and add test results
    metrics_path = os.path.join(ride_dir, "metrics.json")
    with open(metrics_path, "r") as f:
        all_metrics = json.load(f)
    
    # Add test metrics
    all_metrics["test"] = test_metrics
    all_metrics["confidence_intervals"] = ci_metrics
    all_metrics["test_timestamp"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Save updated metrics
    with open(metrics_path, "w") as f:
        json.dump(all_metrics, f, indent=4)
    
    # Save test results
    results_dir = os.path.join(ride_dir, "results")
    os.makedirs(results_dir, exist_ok=True)
    results_df.to_csv(os.path.join(results_dir, "test_results.csv"), index=False)
    
    # Save test evaluation figure
    fig_dir = os.path.join(ride_dir, "figures")
    os.makedirs(fig_dir, exist_ok=True)
    fig.savefig(os.path.join(fig_dir, "test_evaluation.png"), dpi=300, bbox_inches='tight')
    
    print(f"Test results saved for {ride_name}")

## Test Set Evaluation Pipeline

In [15]:
def evaluate_single_ride_on_test(ride_name, test_data, model_dir="models"):
    print(f"\n{'='*50}")
    print(f"Evaluating ride: {ride_name}")
    print(f"{'='*50}")
    
    # Load the saved model
    prophet_ts, existing_metrics = load_model(ride_name, model_dir)
    
    if prophet_ts is None:
        print(f"No saved model found for {ride_name}")
        return None
    
    # Filter test data for the current ride
    ride_test_data = filter_ride_data(test_data, ride_name)
    
    print(f"Test data size: {len(ride_test_data)}")
    
    # Skip if not enough test data
    if len(ride_test_data) < 10:
        print(f"Skipping {ride_name} due to insufficient test data")
        return None
    
    # Prepare test data for prediction
    future_test = prophet_ts.prepare_prophet_dataframe(ride_test_data, include_y=False)
    
    # Generate predictions
    print("Generating predictions on test set...")
    test_forecast = prophet_ts.predict(future_test)
    
    # Merge predictions with original data
    ride_test_data_with_forecast = pd.merge(
        ride_test_data,
        test_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']],
        left_on='time_bucket', 
        right_on='ds',
        how='left'
    )
    
    # Remove any rows with missing predictions
    ride_test_data_with_forecast = ride_test_data_with_forecast.dropna(subset=['yhat'])
    
    if len(ride_test_data_with_forecast) == 0:
        print(f"No valid predictions generated for {ride_name}")
        return None
    
    # Evaluate on test set
    print("Evaluating on test set...")
    test_actuals = ride_test_data_with_forecast['wait_time'].values
    test_predictions = ride_test_data_with_forecast['yhat'].values
    test_lower = ride_test_data_with_forecast['yhat_lower'].values
    test_upper = ride_test_data_with_forecast['yhat_upper'].values
    
    # Evaluate point predictions
    test_metrics, test_results_df, fig_evaluation = evaluate_model(
        ride_test_data_with_forecast, test_actuals, test_predictions, 
        title=f"{ride_name} - Test Set"
    )
    
    # Evaluate confidence intervals
    ci_metrics = evaluate_confidence_intervals(
        test_actuals, test_lower, test_upper, test_predictions
    )
    
    # Add residual analysis
    test_results_df['residual'] = test_results_df['actual'] - test_results_df['predicted']
    test_results_df['abs_residual'] = np.abs(test_results_df['residual'])
    
    # Save results
    save_test_results(ride_name, test_metrics, ci_metrics, test_results_df, 
                     fig_evaluation, model_dir)
    
    # Combine all metrics
    combined_metrics = {
        "test": test_metrics,
        "confidence_intervals": ci_metrics,
        "validation": existing_metrics.get("validation", {}),
        "data_counts": {
            **existing_metrics.get("data_counts", {}),
            "test": len(ride_test_data_with_forecast)
        }
    }
    
    plt.close(fig_evaluation)  # Close the figure to save memory
    
    return combined_metrics

def evaluate_all_rides_on_test(test_data, model_dir="models"):
    # Get all rides that have saved models
    processed_rides = get_processed_rides(model_dir)
    print(f"Found {len(processed_rides)} rides with saved models")
    
    # Initialize results dictionary
    all_test_results = {}
    
    # Process each ride
    for i, ride_name in enumerate(processed_rides):
        print(f"\nEvaluating ride {i+1}/{len(processed_rides)}: {ride_name}")
        
        ride_metrics = evaluate_single_ride_on_test(ride_name, test_data, model_dir)
        
        if ride_metrics:
            all_test_results[ride_name] = ride_metrics
        
        # Print progress
        if (i + 1) % 10 == 0:
            print(f"\nCompleted {i + 1}/{len(processed_rides)} rides")
    
    return all_test_results

## Test Results Analysis and Reporting

In [16]:
def generate_test_summary_report(all_results, output_dir="models"):
    # Create lists to store summary data
    rides = []
    val_mae = []
    val_rmse = []
    val_smape = []
    test_mae = []
    test_rmse = []
    test_smape = []
    test_r2 = []
    coverage_rates = []
    interval_widths = []
    data_counts = []
    
    # Extract data from results
    for ride_name, metrics in all_results.items():
        if not metrics:
            continue
            
        rides.append(ride_name)
        
        # Validation metrics
        val_metrics = metrics.get("validation", {})
        val_mae.append(val_metrics.get("mae", float('nan')))
        val_rmse.append(val_metrics.get("rmse", float('nan')))
        val_smape.append(val_metrics.get("smape", float('nan')))
        
        # Test metrics
        test_metrics = metrics.get("test", {})
        test_mae.append(test_metrics.get("mae", float('nan')))
        test_rmse.append(test_metrics.get("rmse", float('nan')))
        test_smape.append(test_metrics.get("smape", float('nan')))
        test_r2.append(test_metrics.get("r2", float('nan')))
        
        # Confidence interval metrics
        ci_metrics = metrics.get("confidence_intervals", {})
        coverage_rates.append(ci_metrics.get("coverage_rate", float('nan')))
        interval_widths.append(ci_metrics.get("avg_interval_width", float('nan')))
        
        # Data counts
        counts = metrics.get("data_counts", {})
        data_counts.append(f"Val: {counts.get('validation', 0)}, Test: {counts.get('test', 0)}")
    
    # Create DataFrame
    summary_df = pd.DataFrame({
        "Ride Name": rides,
        "Validation MAE": val_mae,
        "Validation RMSE": val_rmse,
        "Validation sMAPE": val_smape,
        "Test MAE": test_mae,
        "Test RMSE": test_rmse,
        "Test sMAPE": test_smape,
        "Coverage Rate (%)": coverage_rates,
        "Avg Interval Width": interval_widths,
        "Data Counts": data_counts
    })
    
    # Sort by test MAE
    summary_df = summary_df.sort_values("Test MAE")
    
    # Save to CSV
    summary_path = os.path.join(output_dir, "test_evaluation_summary.csv")
    summary_df.to_csv(summary_path, index=False)
    
    # Print summary statistics
    print("\n" + "="*80)
    print("TEST SET EVALUATION SUMMARY:")
    print(f"Total rides evaluated: {len(summary_df)}")
    print(f"\nTest Set Performance:")
    print(f"  Average MAE: {np.nanmean(test_mae):.2f} ± {np.nanstd(test_mae):.2f}")
    print(f"  Average RMSE: {np.nanmean(test_rmse):.2f} ± {np.nanstd(test_rmse):.2f}")
    print(f"  Average sMAPE: {np.nanmean(test_smape):.2f}% ± {np.nanstd(test_smape):.2f}%")
    print(f"\nConfidence Intervals:")
    print(f"  Average Coverage Rate: {np.nanmean(coverage_rates):.1f}% (target: 95%)")
    print(f"  Average Interval Width: {np.nanmean(interval_widths):.2f} minutes")
    print(f"\nValidation vs Test Performance:")
    val_test_mae_diff = np.nanmean(test_mae) - np.nanmean(val_mae)
    print(f"  MAE difference (Test - Validation): {val_test_mae_diff:.2f}")
    val_test_rmse_diff = np.nanmean(test_rmse) - np.nanmean(val_rmse)
    print(f"  RMSE difference (Test - Validation): {val_test_rmse_diff:.2f}")
    print(f"\nSummary saved to: {summary_path}")
    print("="*80)
    
    # Create comprehensive visualizations
    create_test_evaluation_plots(summary_df, output_dir)
    
    return summary_df

def create_test_evaluation_plots(summary_df, output_dir):
    # Create a comprehensive plot with multiple subplots
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    
    # 1. Validation vs Test MAE comparison
    valid_data = summary_df.dropna(subset=['Validation MAE', 'Test MAE'])
    axes[0,0].scatter(valid_data['Validation MAE'], valid_data['Test MAE'], alpha=0.6, s=50)
    max_mae = max(valid_data['Validation MAE'].max(), valid_data['Test MAE'].max())
    axes[0,0].plot([0, max_mae], [0, max_mae], 'k--', alpha=0.7)
    axes[0,0].set_xlabel('Validation MAE (minutes)')
    axes[0,0].set_ylabel('Test MAE (minutes)')
    axes[0,0].set_title('Validation vs Test MAE')
    axes[0,0].grid(True, alpha=0.3)
    
    # 2. Test MAE distribution
    axes[0,1].hist(summary_df['Test MAE'].dropna(), bins=20, alpha=0.7, 
                   color=TEST_COLOR, edgecolor='black')
    axes[0,1].axvline(summary_df['Test MAE'].mean(), color='red', linestyle='--', 
                      label=f'Mean: {summary_df["Test MAE"].mean():.1f}')
    axes[0,1].set_xlabel('Test MAE (minutes)')
    axes[0,1].set_ylabel('Number of Rides')
    axes[0,1].set_title('Distribution of Test MAE')
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)
        
    # 4. Coverage rate analysis
    axes[1,0].hist(summary_df['Coverage Rate (%)'].dropna(), bins=20, alpha=0.7, 
                   color=TEST_COLOR, edgecolor='black')
    axes[1,0].axvline(95, color='red', linestyle='--', label='Target: 95%')
    axes[1,0].axvline(summary_df['Coverage Rate (%)'].mean(), color='orange', 
                      linestyle='--', label=f'Mean: {summary_df["Coverage Rate (%)"].mean():.1f}%')
    axes[1,0].set_xlabel('Coverage Rate (%)')
    axes[1,0].set_ylabel('Number of Rides')
    axes[1,0].set_title('Confidence Interval Coverage Rate')
    axes[1,0].legend()
    axes[1,0].grid(True, alpha=0.3)
    
    # 5. Top 15 rides by test performance
    top_rides = summary_df.nsmallest(15, 'Test MAE')
    y_pos = np.arange(len(top_rides))
    axes[1,1].barh(y_pos, top_rides['Test MAE'], color=TEST_COLOR, alpha=0.7)
    axes[1,1].set_yticks(y_pos)
    axes[1,1].set_yticklabels(top_rides['Ride Name'], fontsize=8)
    axes[1,1].set_xlabel('Test MAE (minutes)')
    axes[1,1].set_title('Top 15 Rides by Test Performance')
    axes[1,1].grid(True, alpha=0.3)
    
    # 6. Performance degradation (Test vs Validation)
    perf_change = summary_df['Test MAE'] - summary_df['Validation MAE']
    valid_change = perf_change.dropna()
    axes[1,2].hist(valid_change, bins=20, alpha=0.7, color=TEST_COLOR, edgecolor='black')
    axes[1,2].axvline(0, color='red', linestyle='--', label='No change')
    axes[1,2].axvline(valid_change.mean(), color='orange', linestyle='--',
                      label=f'Mean: {valid_change.mean():.1f}')
    axes[1,2].set_xlabel('MAE Change (Test - Validation)')
    axes[1,2].set_ylabel('Number of Rides')
    axes[1,2].set_title('Performance Change: Test vs Validation')
    axes[1,2].legend()
    axes[1,2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "test_evaluation_comprehensive.png"), 
                dpi=300, bbox_inches='tight')
    plt.close()
    
    # Create a separate plot for model performance ranking
    plt.figure(figsize=(16, 10))
    
    # Sort by test MAE and show top 20
    top_20 = summary_df.nsmallest(20, 'Test MAE')
    
    x_pos = np.arange(len(top_20))
    width = 0.35
    
    plt.bar(x_pos - width/2, top_20['Validation MAE'], width, 
            label='Validation MAE', color=VAL_COLOR, alpha=0.8)
    plt.bar(x_pos + width/2, top_20['Test MAE'], width, 
            label='Test MAE', color=TEST_COLOR, alpha=0.8)
    
    plt.xlabel('Rides (Ranked by Test Performance)')
    plt.ylabel('Mean Absolute Error (minutes)')
    plt.title('Top 20 Rides: Validation vs Test Performance')
    plt.xticks(x_pos, top_20['Ride Name'], rotation=45, ha='right')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    plt.savefig(os.path.join(output_dir, "top_20_rides_comparison.png"), 
                dpi=300, bbox_inches='tight')
    plt.close()

## Main Execution

In [17]:
# Load the data
print("Loading data...")
data = load_data("../data/processed/ep/final_cleaned_processed_wait_times.parquet")
print(f"Loaded data with {len(data)} rows")

check_for_missing_values(data)

data = filter_to_operating_hours(data)

# Define time periods for splitting
train_years, val_year, test_year = list(range(2017, 2023)), 2023, 2024

# Split the data
train_data, val_data, test_data = split_data(data, train_years, val_year, test_year)

# Set model directory
model_dir = "../models/prophet_enhanced/"

# Check if models exist
processed_rides = get_processed_rides(model_dir)
if not processed_rides:
    print(f"No saved models found in {model_dir}")
    print("Please run the training script first to create models.")
else:
    print(f"Found {len(processed_rides)} rides with saved models")
    
    # Evaluate all rides on test set
    print("\nStarting test set evaluation...")
    test_results = evaluate_all_rides_on_test(test_data, model_dir)
    
    # Generate comprehensive summary report
    print("\nGenerating test evaluation summary...")
    test_summary_df = generate_test_summary_report(test_results, model_dir)
    
    print("\nTest evaluation completed successfully!")
    print(f"Results saved to: {model_dir}")

Loading data...
Loaded data with 7834739 rows
No missing values found in the dataset.
Train data size: 297362
Validation data size: 61851
Test data size: 55699
Found 30 rides with saved models

Starting test set evaluation...
Found 30 rides with saved models

Evaluating ride 1/30: volo da vinci

Evaluating ride: volo da vinci
Test data size: 1830
Generating predictions on test set...
Evaluating on test set...

volo da vinci - Test Set MAE: 6.98 minutes
volo da vinci - Test Set RMSE: 8.63 minutes
volo da vinci - Test Set MSE: 74.48
volo da vinci - Test Set sMAPE: 24.99%
Confidence Interval Evaluation:
Coverage Rate: 80.3% (target: 95%)
Average Interval Width: 22.53 minutes
Interval Score: 55.68
Test results saved for volo da vinci

Evaluating ride 2/30: matterhornblitz

Evaluating ride: matterhornblitz
Test data size: 1830
Generating predictions on test set...
Evaluating on test set...

matterhornblitz - Test Set MAE: 18.32 minutes
matterhornblitz - Test Set RMSE: 21.84 minutes
matterho

## Additional Analysis Functions

In [18]:
def analyze_test_results(model_dir="../models/prophet_enhanced/"):
    # Load the test summary results
    summary_path = os.path.join(model_dir, "test_evaluation_summary.csv")
    if not os.path.exists(summary_path):
        print("Test evaluation summary not found. Run the test evaluation first.")
        return None
    
    summary_df = pd.read_csv(summary_path)
    
    # Display top performing rides
    print("TOP 10 RIDES BY TEST PERFORMANCE:")
    print("="*50)
    top_10 = summary_df.head(10)[['Ride Name', 'Test MAE', 'Test RMSE', 'Coverage Rate (%)']]
    print(top_10.to_string(index=False))
    
    # Display bottom performing rides
    print("\nBOTTOM 10 RIDES BY TEST PERFORMANCE:")
    print("="*50)
    bottom_10 = summary_df.tail(10)[['Ride Name', 'Test MAE', 'Test RMSE', 'Coverage Rate (%)']]
    print(bottom_10.to_string(index=False))
    
    # Performance statistics
    print(f"\nOVERALL STATISTICS:")
    print("="*50)
    print(f"Number of rides evaluated: {len(summary_df)}")
    print(f"Test MAE - Mean: {summary_df['Test MAE'].mean():.2f}, Std: {summary_df['Test MAE'].std():.2f}")
    print(f"Test RMSE - Mean: {summary_df['Test RMSE'].mean():.2f}, Std: {summary_df['Test RMSE'].std():.2f}")
    print(f"Test sMAPE - Mean: {summary_df['Test sMAPE'].mean():.4f}, Std: {summary_df['Test sMAPE'].std():.4f}")
    print(f"Coverage Rate - Mean: {summary_df['Coverage Rate (%)'].mean():.1f}%, Target: 95%")
    
    return summary_df

# Run analysis if results exist
if os.path.exists("../models/prophet_enhanced/test_evaluation_summary.csv"):
    test_analysis_df = analyze_test_results()

TOP 10 RIDES BY TEST PERFORMANCE:
                        Ride Name  Test MAE  Test RMSE  Coverage Rate (%)
    madame freudenreich curiosits  0.084556   0.638820          98.526201
whale adventures  northern lights  1.428335   3.015351          86.626638
                    kolumbusjolle  1.542395   2.125228          92.806150
              castello dei medici  1.716850   2.361465          95.275591
                     poppy towers  1.954676   2.466868          97.475302
      old mac donalds tractor fun  2.170635   2.758021          96.369637
                       vindjammer  3.265203   4.038041          85.183130
     vienna wave swing  glckspilz  3.649488   4.327656          73.490670
                        eurotower  3.755417   5.504243          87.883772
                  tirol log flume  4.759194   8.151171          90.639269

BOTTOM 10 RIDES BY TEST PERFORMANCE:
                       Ride Name  Test MAE  Test RMSE  Coverage Rate (%)
                     silver star 13.00499